In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LightModel.light_model import LightModel
from lenstronomy.PointSource.point_source import PointSource
from lenstronomy.Data.pixel_grid import PixelGrid
from lenstronomy.Data.psf import PSF
import lenstronomy.Plots.output_plots as lens_plot
from lenstronomy.ImSim.image_model import ImageModel
from lenstronomy.Data.imaging_data import ImageData
from lenstronomy.Plots.output_plots import LensModelPlot
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver
import lenstronomy.Util.param_util as param_util
import lenstronomy.Util.simulation_util as sim_util
import lenstronomy.Util.image_util as image_util

In [ ]:
#The first source is modeled as a lens since it also deflects the light coming from the second source
lens_model_list = ['SPEP','SPEP']
kwargs_spep_l = {'theta_E': .9, 'e1': 0.05, 'e2': 0.05, 'gamma': 2., 'center_x': 0, 'center_y': 0}
kwargs_spep_s = {'theta_E': .3, 'e1': 0.05, 'e2': 0.05, 'gamma': 1.5, 'center_x': 0, 'center_y': 0}
kwargs_lens = [kwargs_spep_l, kwargs_spep_s]
redshift_list = [0.4, 1.]
z_source = 1.3
lensModel_mp = LensModel(lens_model_list=lens_model_list, z_source=z_source, lens_redshift_list=redshift_list, multi_plane=True)
#source, image position & magnification
beta_ra, beta_dec = 0., 0. #Aligned with the lens 
solver = LensEquationSolver(lensModel_mp)
theta_ra, theta_dec = solver.image_position_from_source(beta_ra, beta_dec, kwargs_lens)
mag = lensModel_mp.magnification(theta_ra, theta_dec, kwargs_lens)
#source light (both sources)
source_light_model_list = ['SERSIC', 'SERSIC']
lightModel_source = LightModel(light_model_list=source_light_model_list)
kwargs_light_source1 = {'amp': 5000, 'R_sersic': 0.05, 'n_sersic': 1.5, 'center_x': beta_ra, 'center_y': beta_dec}
kwargs_light_source2 = {'amp': 3000, 'R_sersic': 0.07, 'n_sersic': 1.5, 'center_x': beta_ra, 'center_y': beta_dec}
kwargs_light_source = [kwargs_light_source1, kwargs_light_source2]
#lens light
lens_light_model_list = ['SERSIC_ELLIPSE']
lightModel_lens = LightModel(light_model_list=lens_light_model_list)
e1, e2 = param_util.phi_q2_ellipticity(phi=0.5, q=0.7)
kwargs_light_l = {'amp': 10000, 'R_sersic': 0.1, 'n_sersic': 2.5, 'e1': e1, 'e2': e2, 'center_x': 0., 'center_y': 0}
kwargs_light_lens = [kwargs_light_l]
#Multi-plane model
kwargs_model_mp = {'lens_model_list': lens_model_list, 'z_source': z_source, 'lens_redshift_list': redshift_list,
                   'multi_plane': True,
                   'source_light_model_list': source_light_model_list,
               'lens_light_model_list': lens_light_model_list, 'point_source_model_list': ['SOURCE_POSITION']}
#point source
point_source_model_list = ['SOURCE_POSITION']
pointSource = PointSource(point_source_type_list=point_source_model_list, lensModel=lensModel_mp, fixed_magnification_list=[True])
kwargs_ps = [{'ra_source': beta_ra, 'dec_source': beta_dec, 'source_amp': 100}]
#Data 
numPix = 100
deltaPix = 0.05 
exp_time = 10
background_rms = .1
ra_at_xy_0, dec_at_xy_0 = -2.5, -2.5 
transform_pix2angle = np.array([[1, 0], [0, 1]]) * deltaPix 
kwargs_pixel = {'nx': 100, 'ny': 100, 'ra_at_xy_0': ra_at_xy_0, 'dec_at_xy_0': dec_at_xy_0, 'transform_pix2angle': transform_pix2angle} 
pixel_grid = PixelGrid(**kwargs_pixel)
#PSF
kwargs_psf = {'psf_type': 'GAUSSIAN', 'fwhm': 0.1, 'pixel_size': deltaPix}
psf = PSF(**kwargs_psf)
#Image data class
kwargs_data = sim_util.data_configure_simple(numPix, deltaPix, exp_time, background_rms, inverse=True)
data_class = ImageData(**kwargs_data)

In [ ]:
kwargs_numerics = {'supersampling_factor': 1, 'supersampling_convolution': False}
imageModel = ImageModel(data_class=pixel_grid, psf_class=psf, lens_model_class=lensModel_mp,
                        source_model_class=lightModel_source, lens_light_model_class=lightModel_lens,
                        point_source_class=pointSource, kwargs_numerics=kwargs_numerics)
image = imageModel.image(kwargs_lens=kwargs_lens, kwargs_source=kwargs_light_source,
                         kwargs_lens_light=kwargs_light_lens, kwargs_ps=kwargs_ps)
#Noise
poisson = image_util.add_poisson(image, exp_time=exp_time)
bkg = image_util.add_background(image, sigma_bkd=background_rms)
image_noisy = image + bkg + poisson
#Image data
kwargs_data = {'image_data': image_noisy, 'background_rms': background_rms, 'exposure_time': exp_time,
               'ra_at_xy_0': ra_at_xy_0, 'dec_at_xy_0': dec_at_xy_0, 'transform_pix2angle': transform_pix2angle} 

In [ ]:
# illustrate fit
kwargs_model = {'lens_model_list': lens_model_list, 'source_light_model_list': source_light_model_list,
               'lens_light_model_list': lens_light_model_list, 'point_source_model_list': point_source_model_list}
lensPlot = LensModelPlot(kwargs_data, kwargs_psf, kwargs_numerics, kwargs_model, kwargs_lens, kwargs_light_source,
                            kwargs_light_lens, kwargs_ps, arrow_size=0.02)
f, axes = plt.subplots(1, 1, figsize=(6, 6), sharex=True, sharey=True)
lensPlot.data_plot(ax=axes)
f.tight_layout()
plt.show()